# Notebook to load scraped data and clean

In [2]:
import pandas as pd
import numpy as np
import string
import datetime
import re

In [608]:
cols = [
    'product',
    'productCategory',
    'productType',
    'price',
    'averageRating',
    'username',
    'userRating',
    'userLocation',
    'athleteType',
    'ageRange',
    'bodyType',
    'likes',
    'dislikes',
    'fit',
    'reviewTitle',
    'reviewDate',
    'reviewText',
    'responseDate',
    'responseText',
    'Helpful',
    'NotHelpful'
]
df = pd.read_csv('mensrunning_lululemon.csv', names=cols)
pd.options.display.max_colwidth = 100

In [609]:
df.tail()

,product,productCategory,productType,price,averageRating,username,userRating,userLocation,athleteType,ageRange,...,likes,dislikes,fit,reviewTitle,reviewDate,reviewText,responseDate,responseText,Helpful,NotHelpful
2413,Metal Vent Tech Long Sleeve\nWool,Tops,Long Sleeves,$88.00,3 out of 5,SFWALSH,5 out of 5,CALGARY,SWEATY GENERALIST,25-34,...,"wool, function",NaN,NaN,The wool makes a difference!,2017-06-29,I own both a metal tech long sleeve with and without wool. While I enjoy lounging around my hous...,NaN,NaN,1,0
2414,Metal Vent Tech Long Sleeve\nWool,Tops,Long Sleeves,$88.00,3 out of 5,MKCROSSFITTER,3 out of 5,OH,SWEATY GENERALIST,25-34,...,fabric color design,sizing,NaN,"Great shirt, strange sizing",2016-12-23,"I bought this shirt in gray at the local store, and I should have tried it on. The fabric is sof...",NaN,NaN,19,2
2415,Pulse Slub Henley\nLightweight Rulu,Tops,Long Sleeves,$88.00,2.3 out of 5,RICKADILLO,5 out of 5,"MELBOURNE, AUSTRALIA",DANCER,25-34,...,NaN,NaN,NaN,Nice!,2018-08-03,Comfy but still form fitting.,NaN,NaN,0,0
2416,Pulse Slub Henley\nLightweight Rulu,Tops,Long Sleeves,$88.00,2.3 out of 5,NORBY515,1 out of 5,"DES MOINES, IA, USA",YOGI,35-44,...,henley,"cut, feel was rough",NaN,Bring back Surge Henley please,2018-01-14,"This shirt does not feel like other Rulu shirts I’ve purchased. It’s rough, not soft and stretch...",NaN,NaN,2,0
2417,Pulse Slub Henley\nLightweight Rulu,Tops,Long Sleeves,$88.00,2.3 out of 5,ELULU91,1 out of 5,"TORONTO, ON, CANADA",RUNNER,25-34,...,"colour, fit","not rulu, not soft, not buttery",NaN,Clearly Not Rulu,2018-01-07,I have been buying Lululemon for nearly 10 years now. This shirt is certainly not RULU. It is no...,NaN,NaN,14,0


In [610]:
## Pre process data ##

# Convert to datetime objects # 
df['reviewDate'] = pd.to_datetime(df['reviewDate'], infer_datetime_format=True)
df['responseDate'] = pd.to_datetime(df['responseDate'], infer_datetime_format=True)

# Strip ratings
df['userRating'] = df['userRating'].apply(lambda x: re.split(' ', x)[0]).map(float)
df['averageRating'] = df['averageRating'].apply(lambda x: re.split(' ', x)[0]).map(float)

# Create columns for length of each review
df['reviewLength'] = df['reviewText'].apply(lambda x: len(x))

# Strip dollar signs
df['price'] = df['price'].str.slice(1).astype(float)

# Create boolean column for response or not
df['response'] = np.where(df['responseText'].notnull(), 1, 0)

In [611]:
# Find products with different product types 

grp_prod = df.groupby(['product', 'productType'])['productType'].describe().reset_index()
mismatched = grp_prod[grp_prod.duplicated(subset='product', keep=False)][['product', 'productType']]
mismatched = mismatched['product'].unique()

In [612]:
# Replace mismatched productType with 'Hoodies'

X = df[ (df['product']==mismatched[0]) | (df['product']==mismatched[1]) ].index
df.loc[X, 'productType'] = 'Hoodies'

In [613]:
# Find duplicate reviews

# Some products have different urls with the same reviews to them if there are styles of the product on sale
# There are also duplicates when a product is updated and reviews stay the same

df = df[-df.duplicated(subset=['reviewText','username'])]

In [614]:
# Missing values

df[df.columns[df.isnull().any()]].isnull().sum()

userLocation     344
athleteType      346
ageRange         311
bodyType         312
likes            773
dislikes         939
fit             1548
reviewTitle        8
responseDate    1190
responseText    1190
dtype: int64

In [615]:
# Replace missing values where a customer wouldn't have responded, i.e. not responseDate and responseText

missing_vals = ['userLocation', 'athleteType', 'ageRange', 'bodyType', 'likes', 'dislikes', 'fit', 'reviewTitle']
df[missing_vals] = df[missing_vals].apply(lambda x: x.fillna('No Response'))
df[df.columns[df.isnull().any()]].isnull().sum()

responseDate    1190
responseText    1190
dtype: int64

In [616]:
df.head()
df.to_csv('cleaned_mensrunning_lululemon.csv', index=False)

In [617]:
df = pd.read_csv('cleaned_mensrunning_lululemon.csv')

In [618]:
df.head()

,product,productCategory,productType,price,averageRating,username,userRating,userLocation,athleteType,ageRange,...,fit,reviewTitle,reviewDate,reviewText,responseDate,responseText,Helpful,NotHelpful,reviewLength,response
0,Metal Vent Tech Short Sleeve,Tops,Short Sleeves,68.0,2.9,NYR26,5.0,"STAMFORD, CT",SWEATY GENERALIST,18-24,...,No Response,The best tshirt,2018-10-12,These are the most comfortable T-shirts. Wish you guys would make more colors and bring back the...,NaN,NaN,0,0,150,0
1,Metal Vent Tech Short Sleeve,Tops,Short Sleeves,68.0,2.9,MJB23,1.0,"CHICAGO, IL, USA",SWEATY GENERALIST,18-24,...,No Response,Very Poor Quality,2018-10-11,I purchased this Short Sleeve expecting it to be just like all the other ones I have in my colle...,2018-10-11,"Dear mjb23,\n\nThanks for reaching out and providing this feedback for us. We hear where you're ...",0,0,567,1
2,Metal Vent Tech Short Sleeve,Tops,Short Sleeves,68.0,2.9,JKD123,1.0,"CHICAGO, IL, USA",RUNNER,25-34,...,No Response,Stretches When You Sweat,2018-10-08,I bought this shirt for my husband to run the Chicago Marathon. Upon starting the race it fit gr...,2018-10-09,"Hi Jkd123,\n\nThanks for reaching out and providing this feedback for us. I completely understan...",0,0,269,1
3,Metal Vent Tech Short Sleeve,Tops,Short Sleeves,68.0,2.9,SLASH,5.0,"OVERLAND PARK, KS, USA",SWEATY GENERALIST,45-54,...,No Response,Love Lulu,2018-10-08,I have several shirts and shorts I have purchased from Lululemon. Hands down the best clothing I...,NaN,NaN,0,0,170,0
4,Metal Vent Tech Short Sleeve,Tops,Short Sleeves,68.0,2.9,BRANDONM19,4.0,"HUTCHINSON, KS",SWEATY GENERALIST,18-24,...,No Response,Sweat wicking shirt,2018-10-03,Great for workouts.,NaN,NaN,0,0,19,0
